# 05 Grouping – Solutions

## 1. Reload the `planes` and `flights` DataFrames, from the last lab.

In [1]:
import pandas as pd
planes = pd.read_csv('../data/planes.csv')
flights = pd.read_csv('../data/flights.csv')

## 2. What is the average departure delay (`dep_delay`) of all flights in this data?

In [2]:
flights['dep_delay'].mean()

12.639070257304708

## 3. What is the average departure delay by carrier, for flights in this data?

In [5]:
flights.groupby('carrier').agg({'dep_delay': ['mean']})

,dep_delay
,mean
carrier,
9E,16.725769
AA,8.586016
AS,5.804775
B6,13.022522
DL,9.264505
EV,19.955390
F9,20.215543
FL,18.726075


## 4. Try removing the brackets and rerunning #3. What's different about the result? Why do you think we focused on the list-based approach in class?

In [6]:
flights.groupby('carrier').agg({'dep_delay': 'mean'})

,dep_delay
carrier,
9E,16.725769
AA,8.586016
AS,5.804775
B6,13.022522
DL,9.264505
EV,19.955390
F9,20.215543
FL,18.726075
HA,4.900585


Without the brackets, the result just has a different column index (column naming format, basically).
When we pass in a list (as in #3), we get back a 2-level column index; the first level being the column we're aggregating and the second being how we're aggregating it.
This is because a list (as in #3) *could* include multiple metrics, so Pandas sets up our result DataFrame in a way that could accommodate those multiple metrics.

As with most things in Pandas, groupbys can be done in many different ways.
In class, we looked specifically at this format because it's the most **extensible** -- it allows the user to pass multiple columns to aggregate and multiple aggregations per column if they'd like, all using the same basic pattern.

The *easiest* but least extensible style of doing a groupby is below:

In [7]:
flights.groupby('carrier')['dep_delay'].mean()

carrier
9E    16.725769
AA     8.586016
AS     5.804775
B6    13.022522
DL     9.264505
EV    19.955390
F9    20.215543
FL    18.726075
HA     4.900585
MQ    10.552041
OO    12.586207
UA    12.106073
US     3.782418
VX    12.869421
WN    17.711744
YV    18.996330
Name: dep_delay, dtype: float64

## 5. Working from your code for #3, calculate the minimum, mean, median, and maximum departure delay for each carrier.

In [8]:
flights.groupby('carrier').agg({'dep_delay': ['min', 'mean', 'median', 'max']})

dep_delay                          
              min       mean median     max
carrier                                    
9E          -24.0  16.725769   -2.0   747.0
AA          -24.0   8.586016   -3.0  1014.0
AS          -21.0   5.804775   -3.0   225.0
B6          -43.0  13.022522   -1.0   502.0
DL          -33.0   9.264505   -2.0   960.0
EV          -32.0  19.955390   -1.0   548.0
F9          -27.0  20.215543    0.5   853.0
FL          -22.0  18.726075    1.0   602.0
HA          -16.0   4.900585   -4.0  1301.0
MQ          -26.0  10.552041   -3.0  1137.0
OO          -14.0  12.586207   -6.0   154.0
UA          -20.0  12.106073    0.0   483.0
US          -19.0   3.782418   -4.0   500.0
VX          -20.0  12.869421    0.0   653.0
WN          -13.0  17.711744    1.0   471.0
YV          -16.0  18.996330   -2.0   387.0

## 6. Build a groupby-agg invocation that summarizes – at the carrier level – the average departure delay and the median air time.

In [9]:
flights.groupby('carrier').agg({'dep_delay': ['mean'], 'air_time': ['median']})

,dep_delay,air_time
,mean,median
carrier,,
9E,16.725769,83.0
AA,8.586016,169.0
AS,5.804775,324.0
B6,13.022522,142.0
DL,9.264505,145.0
EV,19.955390,87.0
F9,20.215543,229.0
FL,18.726075,109.0


## 7. What is the single most common route in this data?

This requires you to think about how groupbys can be used.
In this case, we could group by route (`origin` and `dest` columns) and *count* how many times each occurs, then *sort* that data to see which is most common.

In [12]:
# Notice that I chose to count tailnum -- you have to count an individual column,
# so it's best to choose a column that won't have NaNs; Pandas doesn't count NaNs
# by default.
routes = flights.groupby(['origin', 'dest']).agg({'tailnum': ['count']})
routes.head()

tailnum
              count
origin dest        
EWR    ALB      439
       ANC        8
       ATL     5022
       AUS      961
       AVL      265

This result has a 2-level column index, which we haven't worked with much.
To give a 2-level column name as an argument to Pandas, use a tuple:
`('tailnum', 'count')`

In [14]:
# Use ascending=False to sort *descending*
sorted_routes = routes.sort_values(by=('tailnum', 'count'), ascending=False)
sorted_routes.head()

tailnum
              count
origin dest        
JFK    LAX    11237
LGA    ATL    10262
       ORD     8717
JFK    SFO     8174
LGA    CLT     6114

So the most common route is **JFK to LAX**, with 11,237 flights.

## Challenge question: What route is most common for each carrier?

There are many ways to attack this, but almost all of them require multiple steps.
I'm going to do this:
- Count flights by carrier, origin, and dest
- Find how many flights are in the most common route for each carrier.
- Join this "max" number back to the aggregated table to pull out the full records (origin-dest-n_flights) for the most common routes of each carrier.

To start, group by the relevant columns.
Two things to note about the below:
1. I'm using the easier syntax (not `.agg`)
2. `as_index=False` keeps my grouped columns as regular columns in my DF, which makes it easier when I need to do more with them.

In [19]:
carrier_routes = flights.groupby(['carrier', 'origin', 'dest'], as_index=False)['tailnum'].count()
carrier_routes.head()

,carrier,origin,dest,tailnum
0,9E,EWR,ATL,4
1,9E,EWR,CVG,803
2,9E,EWR,DTW,220
3,9E,EWR,MSP,173
4,9E,JFK,ATL,53


Rename the "tailnum" column to "n_flights" for clarity.

In [20]:
carrier_routes = carrier_routes.rename(columns={'tailnum': 'n_flights'})
carrier_routes.head()

,carrier,origin,dest,n_flights
0,9E,EWR,ATL,4
1,9E,EWR,CVG,803
2,9E,EWR,DTW,220
3,9E,EWR,MSP,173
4,9E,JFK,ATL,53


We can easily use groupby to figure out what the largest number of flights for any route is, by carrier.

In [23]:
max_routes_by_carrier = carrier_routes.groupby('carrier', as_index=False)['n_flights'].max()
max_routes_by_carrier

,carrier,n_flights
0,9E,1034
1,AA,5684
2,AS,714
3,B6,3304
4,DL,5544
5,EV,2529
6,F9,682
7,FL,2337
8,HA,342
9,MQ,3334


Now for the interesting part – since we have a table of carrier and n_flights for the most common route for each carrier, we can **left join this back to the carrier_routes table to extract only rows that have those values** (the maximum number of flights).

(In this case, because all rows in the max_routes table are in the other table, an inner join would work just as well).

In [25]:
joined = pd.merge(max_routes_by_carrier, carrier_routes, on=('carrier', 'n_flights'), how='left')
joined

,carrier,n_flights,origin,dest
0,9E,1034,JFK,MSP
1,AA,5684,LGA,ORD
2,AS,714,EWR,SEA
3,B6,3304,JFK,MCO
4,DL,5544,LGA,ATL
5,EV,2529,EWR,DTW
6,F9,682,LGA,DEN
7,FL,2337,LGA,ATL
8,HA,342,JFK,HNL
9,MQ,3334,LGA,RDU


And there you have it -- the most common route for carrier 9E is JFK to MSP.